# Custom Chatbot Project

### **The reason I chose this dataset (apart from the fact that I love movies and plays) is that it's usually cumbersome to go through the entire movie info and get specific things I want to be answered. I see this as something that can be expanded on something like the IMDB dataset - making a chatbot that could help get quick information about movies and help people make decisions faster.**

In [1]:
# Imports
import ast
import openai
import tiktoken
import pandas as pd
from openai.embeddings_utils import get_embedding, distances_from_embeddings

# Constants
batch_size = 100
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
openai.api_key = "YOUR API KEY"

## Data Wrangling

In [6]:
# read in data
df = pd.read_csv("data/character_descriptions.csv")
# create a column with more information
df['text'] = df.apply(lambda row: f"{row['Name']} is a character in a {row['Medium']} set in {row['Setting']}. {row['Description']}", axis=1)
df = df[['text']]

In [7]:
print(df.shape)
# Setting display options for DataFrame
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_rows', None)
df

(55, 1)


,text
0,"Emily is a character in a Play set in England. A young woman in her early 20s, Emily is an aspiring actress and Alice's daughter. She has a bubbly personality and a quick wit, but struggles with self-doubt and insecurity. She's also in a relationship with George."
1,"Jack is a character in a Play set in England. A middle-aged man in his 40s, Jack is a successful businessman and Sarah's boss. He has a no-nonsense attitude, but is fiercely loyal to his friends and family. He's married to Alice."
2,"Alice is a character in a Play set in England. A woman in her late 30s, Alice is a warm and nurturing mother of two, including Emily. She's kind-hearted and empathetic, but can be overly protective of her children and prone to worrying. She's married to Jack."
3,"Tom is a character in a Play set in England. A man in his 50s, Tom is a retired soldier and John's son. He has a no-nonsense approach to life, but is haunted by his experiences in combat and struggles with PTSD. He's also in a relationship with Rachel."
4,"Sarah is a character in a Play set in England. A woman in her mid-20s, Sarah is a free-spirited artist and Jack's employee. She's creative, unconventional, and passionate about her work. However, she can also be flighty and impulsive at times."
5,"George is a character in a Play set in England. A man in his early 30s, George is a charming and charismatic businessman who is in a relationship with Emily. He's ambitious, confident, and always looking for the next big opportunity. However, he's also prone to bending the rules to get what he wants."
6,"Rachel is a character in a Play set in England. A woman in her late 20s, Rachel is a shy and introverted librarian who is in a relationship with Tom. She's intelligent, thoughtful, and has a deep love of books. However, she struggles with social anxiety and often feels like an outsider."
7,"John is a character in a Play set in England. A man in his 60s, John is a retired professor and Tom's father. He has a dry wit and a love of intellectual debate, but can also be stubborn and set in his ways."
8,"Maria is a character in a Movie set in Texas. A middle-aged Latina woman in her 40s, Maria is a hard-working single mother who owns a small family-run diner in a small Texas town. She's fiercely protective of her teenage daughter, Sofia, and is always trying to balance work and family."
9,"Caleb is a character in a Movie set in Texas. A young African American man in his early 20s, Caleb is a talented musician who dreams of making it big in the music industry. He's charismatic, confident, and has a way with words. However, he's also struggling with addiction and is constantly at odds with his strict religious family."


In [8]:
## generate embeddings
embeddings = []

for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,"Emily is a character in a Play set in England. A young woman in her early 20s, Emily is an aspiring actress and Alice's daughter. She has a bubbly personality and a quick wit, but struggles with self-doubt and insecurity. She's also in a relationship with George.","[-0.021480176597833633, -0.010991205461323261, -0.01598135381937027, -0.02571697346866131, -0.04280582070350647, 0.028022101148962975, -0.00543121388182044, 0.019831817597150803, -0.003737782593816519, -0.012916437350213528, -0.00761078204959631, -0.009896592237055302, 0.011770312674343586, -0.0010334437247365713, -0.0007939970819279552, 0.01902051642537117, 0.024995816871523857, -0.01146768406033516, 0.008673201315104961, -0.024313293397426605, -0.012156646698713303, -0.001264439313672483, 0.00925914105027914, -0.0038021716754883528, -0.02706914395093918, -0.003589687868952751, 0.02424890547990799, -0.017642591148614883, 0.0031035507563501596, -0.03724261000752449, -0.011242322623729706, -0.01139041781425476, -0.018067559227347374, -0.03147335350513458, -0.02209831215441227, -0.010759404860436916, -0.018698571249842644, -0.025188984349370003, 0.03268386796116829, 0.013972417451441288, 0.01149343978613615, 0.019973473623394966, -0.017771368846297264, -0.0054891640320420265, -0.012942193076014519, 0.005173657555133104, -0.03546547144651413, -0.0012620247434824705, -0.016045743599534035, 0.027094898745417595, 0.013141798786818981, 0.024274660274386406, -0.014899618923664093, -0.005614722613245249, -0.01905914954841137, 0.009098168462514877, -0.0027703375089913607, -0.0025417564902454615, 0.007681610062718391, 0.0021393252536654472, 0.005746719893068075, 0.002657656790688634, -0.022536156699061394, -0.03355955705046654, -0.0048645902425050735, 0.0018865983001887798, -0.02732669934630394, 0.005708086770027876, 0.015479120425879955, 0.015607898123562336, 0.02897505834698677, 0.012942193076014519, -5.0555441703181714e-05, 0.01372773852199316, 0.02272932417690754, 0.0007992287282831967, -0.01683128997683525, 0.01851828210055828, -0.015968477353453636, 0.00356393214315176, 0.005524578038603067, -0.030546151101589203, 0.0040468499064445496, 0.005086732562631369, 0.01572379842400551, 0.03482158109545708, -0.01366334967315197, 0.03268386796116829, -0.0068703084252774715, 0.018376626074314117, 0.002652827650308609, 0.03188544139266014, 0.02446782775223255, 0.014848107472062111, 0.010791599750518799, -0.0076043433509767056, 5.1083632570225745e-05, 0.04375877603888512, 0.001241098390892148, -0.0284599456936121, ...]"
1,"Jack is a character in a Play set in England. A middle-aged man in his 40s, Jack is a successful businessman and Sarah's boss. He has a no-nonsense attitude, but is fiercely loyal to his friends and family. He's married to Alice.","[0.0016696855891495943, -0.023069651797413826, -0.004641270264983177, -0.033406730741262436, -0.03975999727845192, 0.01821356825530529, -0.0029439192730933428, 0.004585939459502697, 0.0102394362911582, -0.013669940643012524, 0.0002733988221734762, -0.0019251833437010646, 0.009503863751888275, -0.0015728569123893976, 0.016703365370631218, 0.006148218642920256, 0.03356295824050903, -0.018890555948019028, 0.02562137506902218, -0.03632298484444618, -0.018031302839517593, 0.007700733374804258, -3.3717151382006705e-05, -0.02097359485924244, -0.027053464204072952, -0.0054614669643342495, 0.005786941386759281, -0.007140916772186756, 0.020296607166528702, -0.024410609155893326, -0.0030008775647729635, 0.010864348150789738, -0.006691761314868927, -0.014841649681329727, -0.024684006348252296, 0.009458296932280064, -0.015518637374043465, -0.01465938426554203, 0.007316673174500465, 0.002603798173367977, 0.004182350821793079, 0.019971132278442383, -0.004149803426116705, -0.014841649681329727, -0.0013360739685595036, 0.013012481853365898, -0.01831771992146969, -0.030334249138832092, -0.008221493102610111, 0.011339541524648666, -0.004159567877650261, 0.03197464346885681, -0.010786234401166439, -0.009106784127652645, 0.00552656175568

In [9]:
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')
df.to_csv("embeddings.csv")

## Custom Query Completion

In [2]:
df = pd.read_csv("embeddings.csv", index_col=0)
df['embeddings'] = df['embeddings'].apply(ast.literal_eval)

In [3]:
def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy


In [4]:
def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know". 
The data contains character descriptions from theater, television, and film productions. 
Each row contains the name, description, medium, and setting for each character.

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)


def custom_question_answer(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    custom_prompt = create_prompt(question, df, max_prompt_tokens)
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=custom_prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

In [5]:
def simple_question_answer(question):
    simple_prompt = f"""
Question: "{question}"
Answer:
"""
    return openai.Completion.create(
        model=COMPLETION_MODEL_NAME,
        prompt=simple_prompt,
        max_tokens=150
    )["choices"][0]["text"].strip()

## Custom Performance Demonstration

### Question 1

In [7]:
question = "Who all are characters from England?"
simple_answer = simple_question_answer(question)
custom_answer = custom_question_answer(question, df)
print(f"Question: {question}\n\nSimple Answer:\n{simple_answer}\n\nCustom Answer:\n{custom_answer}")

Question: Who all are characters from England?

Simple Answer:
1. Sherlock Holmes 
2. James Bond 
3. Harry Potter 
4. Elizabeth Bennett 
5. Mr. Darcy 
6. Mary Poppins 
7. Mr. Bean 
8. Alice in Wonderland 
9. Oliver Twist 
10. Robin Hood 
11. Peter Pan 
12. Jane Eyre 
13. Dracula 
14. Ebenezer Scrooge 
15. Willy Wonka.

Custom Answer:
Emily, George, John, Jack, Alice, Tom, Sarah, Rachel


### Question 2

In [8]:
question = "What are the different locations that the characters are in?"
simple_answer = simple_question_answer(question)
custom_answer = custom_question_answer(question, df)
print(f"Question: {question}\n\nSimple Answer:\n{simple_answer}\n\nCustom Answer:\n{custom_answer}")

Question: What are the different locations that the characters are in?

Simple Answer:
There is not enough context to determine the specific characters and locations being referred to. However, some common locations where characters may be found include:

1. Home or residence: This could include a character's house, apartment, or any other place they live in.
2. Work or school: Many characters may spend a significant amount of time at their workplace or school.
3. Public places: Characters may be seen in various public places such as parks, cafes, malls, etc.
4. Travel destinations: Characters may go on trips or vacations to different locations such as beaches, mountains, or other countries.
5. The setting of the story: Depending on the plot of the story, characters may be in specific locations such as a medieval castle,

Custom Answer:
The characters are in England, Texas, USA, Australia, and Ancient Greece.


### Question 3

In [9]:
question = "Can you give all distinct medium and settings from the data?"
simple_answer = simple_question_answer(question)
custom_answer = custom_question_answer(question, df)
print(f"Question: {question}\n\nSimple Answer:\n{simple_answer}\n\nCustom Answer:\n{custom_answer}")

Question: Can you give all distinct medium and settings from the data?

Simple Answer:
1. Watercolor on paper 
2. Oil on canvas 
3. Charcoal on paper 
4. Acrylic on canvas 
5. Ink on paper 
6. Mixed media on canvas 
7. Pen and ink on paper 
8. Pencil on paper 
9. Digital print on paper 
10. Screen print on fabric 
11. Etching on paper 
12. Woodcut on paper 
13. Collage on paper 
14. Photography on canvas 
15. Ceramic sculpture 
16. Bronze sculpture 
17. Stone sculpture 
18. Glass blowing 
19. Encaustic painting on wood 
20. Graffiti on concrete.

Custom Answer:
The distinct mediums are Musical, Play, Movie, Reality Show, Limited Series, and Opera. The distinct settings are USA, England, Italy, and Australia.


# Result

### ***From these, we can see that using the custom prompt with the embeddings are much better at answering questions about the dataset, although some of them are not completely correct.***